In [4]:
%pip install requests
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

## 2. Pendahuluan Scrapping Web

In [3]:
url = "https://www.beautyhaul.com/brand/somethinc&sort=date-desc&view=24&fc=138&fb=397"
response = requests.get(url)

#print(response.content)
#print(response)

html = BeautifulSoup(response.content, 'html.parser')

df_products = pd.DataFrame()
product_list = []

products = html.find_all("div", class_="col-d2 col-md-d3 col-lg-d4 product-item product-item-sale")
#print(len(products))

for product in products:
    #print ("-----------------")
    product_brand = product.find("div", class_="brand").get_text()
    product_title= product.find("div", class_="title").get_text()
    product_price = product.find("div", class_="price").find_all("span")
    
    # print("Nama Brand : ", product_brand)
    # print("Nama produk : ", product_title)
    
    if len(product_price) == 2:
        product_discounted_price = product_price[1].get_text()  # Get the second price (discount price)
        # print("Discounted Price: ", product_discounted_price)
        # print("Harga Reguler: ", product_regular_price)
        # print("Harga Diskon : ", product_discounted_price)

    link_product = product.find("a")['href']
    full_link = f"https://www.beautyhaul.com/brand/somethinc&sort=date-desc&view=24&fc=138&fb=397{link_product}"
    product_list.append([product_brand, product_title, product_discounted_price, full_link])
    df_products = pd.DataFrame(product_list, columns=['Brand', 'Product title', 'Price', 'Link'])
    # print("link product : ", link_product)
    # print ("-----------------")
df_products

,Brand,Product title,Price,Link
0,SOMETHINC,Brow Pro Pomade 18HR Stay,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...
1,SOMETHINC,Brow Game Eyebrow Powder Palette,Rp 144.000,https://www.beautyhaul.com/brand/somethinc&sor...
2,SOMETHINC,Brow Emblem Microcara Brow Gel,Rp 74.700,https://www.beautyhaul.com/brand/somethinc&sor...
3,SOMETHINC,Brow Matters,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...
4,SOMETHINC,BROW WIZ Retractable Eyebrow,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...


## 3. Ekstraksi Data Ulasan dan pengangan pagnitation

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.beautyhaul.com/brand/somethinc&sort=date-desc&view=24&fc=138&fb=397"
response = requests.get(url)

html = BeautifulSoup(response.content, 'html.parser')

df_products = pd.DataFrame()
product_list = []

products = html.find_all("div", class_="col-d2 col-md-d3 col-lg-d4 product-item product-item-sale")

for product in products:
    product_brand = product.find("div", class_="brand").get_text()
    product_title = product.find("div", class_="title").get_text()
    product_price = product.find("div", class_="price").find_all("span")

    if len(product_price) == 2:
        product_discounted_price = product_price[1].get_text()
    else:
        product_discounted_price = None

    link_product = product.find("a")['href']
    full_link = f"https://www.beautyhaul.com/brand/somethinc&sort=date-desc&view=24&fc=138&fb=397{link_product}"
    
    # Mengekstrak jumlah ulasan
    review_count_div = product.find("div", class_="review-stars-tx")
    review_count = review_count_div.get_text(strip=True) if review_count_div else None

    product_list.append([product_brand, product_title, product_discounted_price, full_link, review_count])

# Menyimpan ke dalam DataFrame
df_products = pd.DataFrame(product_list, columns=['Brand', 'Product title', 'Price', 'Link', 'Jumlah Ulasan'])
df_products



,Brand,Product title,Price,Link,Jumlah Ulasan
0,SOMETHINC,Brow Pro Pomade 18HR Stay,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...,(53)
1,SOMETHINC,Brow Game Eyebrow Powder Palette,Rp 144.000,https://www.beautyhaul.com/brand/somethinc&sor...,(63)
2,SOMETHINC,Brow Emblem Microcara Brow Gel,Rp 74.700,https://www.beautyhaul.com/brand/somethinc&sor...,(161)
3,SOMETHINC,Brow Matters,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...,(150)
4,SOMETHINC,BROW WIZ Retractable Eyebrow,Rp 58.500,https://www.beautyhaul.com/brand/somethinc&sor...,(281)


## 4. Pembersihan dan Penyimpanan Data:

In [5]:
# Fungsi untuk mengambil ulasan dari satu halaman
def scrape_reviews_from_page(url):
    response = requests.get(url)
    html = BeautifulSoup(response.content, 'html.parser')
    
    reviews = []
    
    # Mengekstrak informasi ulasan
    review_divs = html.find_all("div", class_="review-div")
    for review_div in review_divs:
        rating = len(review_div.find("div", class_="review-stars review-stars-sm").find_all("span", class_="a"))
        review_count = review_div.find("div", class_="review-stars-tx").get_text(strip=True)
        
        reviews.append({
            "Rating": rating,
            "Ulasan": review_count
        })
    
    return reviews

# URL produk yang ingin di-scrape ulasannya
base_url = "https://www.beautyhaul.com/brand/somethinc&sort=date-desc&view=24&fc=138&fb=397"
response = requests.get(base_url)
html = BeautifulSoup(response.content, 'html.parser')

# Mengekstrak jumlah halaman ulasan
pagination = html.find("ul", class_="pagination")
if pagination:
    last_page = int(pagination.find_all("li")[-2].get_text(strip=True))
else:
    last_page = 1

# Mengumpulkan ulasan dari setiap halaman
all_reviews = []
for page_number in range(1, last_page + 1):
    page_url = f"{base_url}&page={page_number}"
    reviews_from_page = scrape_reviews_from_page(page_url)
    all_reviews.extend(reviews_from_page)

# Menyimpan ulasan ke dalam DataFrame
df_reviews = pd.DataFrame(all_reviews)
df_reviews.to_csv("product_reviews.csv", index=False)

# Membersihkan data yang tidak diperlukan
df_reviews["Ulasan"] = df_reviews["Ulasan"].str.strip()